In [1]:
from gurobipy import Model, GRB, quicksum
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus
import pandas as pd

# Question 1

In [2]:
# Load datasets
farms_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/farms.csv')
processing_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/processing.csv')
centers_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/centers.csv')

# Part b

In [3]:
# Extract key data
farm_ids = farms_data['Farm_ID'].tolist()
plant_ids = processing_data['Processing_Plant_ID'].tolist()
center_ids = centers_data['Center_ID'].tolist()

# Define capacities and demands
farm_capacities = dict(zip(farm_ids, farms_data['Bio_Material_Capacity_Tons']))
processing_capacities = dict(zip(plant_ids, processing_data['Capacity_Tons']))
center_demands = dict(zip(center_ids, centers_data['Requested_Demand_Tons']))

# Extract costs as matrices
farm_to_plant_costs = farms_data.iloc[:, 5:5 + 18].values
print(f"Corrected shape of farm_to_plant_costs: {farm_to_plant_costs.shape}")
plant_to_center_costs = processing_data.iloc[:, 5:5 + len(center_ids)].values
processing_costs = dict(zip(plant_ids, processing_data['Processing_Cost_Per_Ton']))


Corrected shape of farm_to_plant_costs: (249, 17)


In [4]:
# Fixing arrays for farm to plant and plant to center:
if farm_to_plant_costs.shape[1] < 18:
    missing_cols = 18 - farm_to_plant_costs.shape[1]
    for i in range(missing_cols):
        farms_data[f"Plant_{farm_to_plant_costs.shape[1] + i + 1}"] = farms_data.iloc[:, 5:5 + farm_to_plant_costs.shape[1]].mean(axis=1)
    farm_to_plant_costs = farms_data.iloc[:, 5:5 + 18].values

if plant_to_center_costs.shape[1] < len(center_ids):
    missing_cols = len(center_ids) - plant_to_center_costs.shape[1]
    for i in range(missing_cols):
        processing_data[f"Missing_Center_{i+1}"] = processing_data.iloc[:, 5:5 + plant_to_center_costs.shape[1]].mean(axis=1)
    plant_to_center_costs = processing_data.iloc[:, 5:5 + len(center_ids)].values



In [5]:
print(f"Shape of farm_to_plant_costs: {farm_to_plant_costs.shape}")
print(f"Number of plant_ids: {len(plant_ids)}")
print(f"Farm IDs: {farm_ids}")
print(f"Plant IDs: {plant_ids}")

Shape of farm_to_plant_costs: (249, 18)
Number of plant_ids: 18
Farm IDs: ['Farm_1', 'Farm_2', 'Farm_3', 'Farm_4', 'Farm_5', 'Farm_6', 'Farm_7', 'Farm_8', 'Farm_9', 'Farm_10', 'Farm_11', 'Farm_12', 'Farm_13', 'Farm_14', 'Farm_15', 'Farm_16', 'Farm_17', 'Farm_18', 'Farm_19', 'Farm_20', 'Farm_21', 'Farm_22', 'Farm_23', 'Farm_24', 'Farm_25', 'Farm_26', 'Farm_27', 'Farm_28', 'Farm_29', 'Farm_30', 'Farm_31', 'Farm_32', 'Farm_33', 'Farm_34', 'Farm_35', 'Farm_36', 'Farm_37', 'Farm_38', 'Farm_39', 'Farm_40', 'Farm_41', 'Farm_42', 'Farm_43', 'Farm_44', 'Farm_45', 'Farm_46', 'Farm_47', 'Farm_48', 'Farm_49', 'Farm_50', 'Farm_51', 'Farm_52', 'Farm_53', 'Farm_54', 'Farm_55', 'Farm_56', 'Farm_57', 'Farm_58', 'Farm_59', 'Farm_60', 'Farm_61', 'Farm_62', 'Farm_63', 'Farm_64', 'Farm_65', 'Farm_66', 'Farm_67', 'Farm_68', 'Farm_69', 'Farm_70', 'Farm_71', 'Farm_72', 'Farm_73', 'Farm_74', 'Farm_75', 'Farm_76', 'Farm_77', 'Farm_78', 'Farm_79', 'Farm_80', 'Farm_81', 'Farm_82', 'Farm_83', 'Farm_84', 'Farm_85',

In [6]:
# Create the optimization model
model = Model("BioAgri_Solutions")

# Decision variables
x_farm_to_plant = model.addVars(
    farm_ids, plant_ids, name="x_farm_to_plant", lb=0, vtype=GRB.CONTINUOUS)
x_plant_to_center = model.addVars(
    plant_ids, center_ids, name="x_plant_to_center", lb=0, vtype=GRB.CONTINUOUS)

# Objective: Minimize total cost
model.setObjective(
    quicksum(x_farm_to_plant[f, p] * farm_to_plant_costs[farm_ids.index(f), plant_ids.index(p)]
             for f in farm_ids for p in plant_ids)
    + quicksum(x_plant_to_center[p, c] * plant_to_center_costs[plant_ids.index(p), center_ids.index(c)]
               for p in plant_ids for c in center_ids)
    + quicksum(x_farm_to_plant[f, p] * processing_costs[p]
               for f in farm_ids for p in plant_ids),
    GRB.MINIMIZE)


Set parameter Username
Set parameter LicenseID to value 2612728


Academic license - for non-commercial use only - expires 2026-01-21


In [7]:
# Constraints
# 1. Farm capacity
model.addConstrs((quicksum(x_farm_to_plant[f, p] for p in plant_ids) <= farm_capacities[f]
                  for f in farm_ids), name="Farm_Capacity")

# 2. Processing plant capacity
model.addConstrs((quicksum(x_farm_to_plant[f, p] for f in farm_ids) <= processing_capacities[p]
                  for p in plant_ids), name="Plant_Capacity")

# 3. Home center demand
model.addConstrs((quicksum(x_plant_to_center[p, c] for p in plant_ids) >= center_demands[c]
                  for c in center_ids), name="Center_Demand")

# 4. Flow conservation at processing plants
model.addConstrs(
    quicksum(x_farm_to_plant[f, p] for f in farm_ids) == quicksum(x_plant_to_center[p, c] for c in center_ids)
    for p in plant_ids
)

# Solve the model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x7822a7dd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.03s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.902000e+04   0.000000e+00      0s
     418    2.2572950e+05   0.000000e+00   0.000000e+00      0s

Solved in 418 iterations and 0.06 seconds (0.02 work units)
Optimal objective  2.257294998e+05


In [8]:
# Output optimized decision variables and total cost
if model.status == GRB.OPTIMAL:
    print(f"Minimum Total Cost: {model.objVal}")
    for f in farm_ids:
        for p in plant_ids:
            if x_farm_to_plant[f, p].x > 0:
                print(f"Farm {f} -> Plant {p}: {x_farm_to_plant[f, p].x} tons")

    for p in plant_ids:
        for c in center_ids:
            if x_plant_to_center[p, c].x > 0:
                print(f"Plant {p} -> Center {c}: {x_plant_to_center[p, c].x} tons")


Minimum Total Cost: 225729.49984810984
Farm Farm_1 -> Plant Plant_2: 173.0 tons
Farm Farm_3 -> Plant Plant_9: 516.0 tons
Farm Farm_8 -> Plant Plant_9: 360.0 tons
Farm Farm_13 -> Plant Plant_1: 562.0 tons
Farm Farm_14 -> Plant Plant_2: 526.0 tons
Farm Farm_18 -> Plant Plant_9: 525.0 tons
Farm Farm_19 -> Plant Plant_9: 229.0 tons
Farm Farm_20 -> Plant Plant_10: 558.0 tons
Farm Farm_22 -> Plant Plant_10: 285.0 tons
Farm Farm_25 -> Plant Plant_16: 512.0 tons
Farm Farm_35 -> Plant Plant_10: 212.0 tons
Farm Farm_36 -> Plant Plant_9: 519.0 tons
Farm Farm_37 -> Plant Plant_2: 550.0 tons
Farm Farm_38 -> Plant Plant_6: 379.0 tons
Farm Farm_42 -> Plant Plant_9: 470.0 tons
Farm Farm_51 -> Plant Plant_10: 422.0 tons
Farm Farm_52 -> Plant Plant_1: 317.0 tons
Farm Farm_53 -> Plant Plant_10: 391.0 tons
Farm Farm_67 -> Plant Plant_9: 291.0 tons
Farm Farm_75 -> Plant Plant_9: 277.0 tons
Farm Farm_79 -> Plant Plant_10: 452.0 tons
Farm Farm_80 -> Plant Plant_9: 212.0 tons
Farm Farm_81 -> Plant Plant_2: 49

# Part c

In [9]:
# Loading datasets again
farms_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/farms.csv')
processing_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/processing.csv')
centers_data = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/centers.csv')

In [10]:
# Print column names to debug KeyError
print("Centers Dataset Columns:", centers_data.columns)
print("Farms Dataset Columns:", farms_data.columns)
print("Processing Dataset Columns:", processing_data.columns)

Centers Dataset Columns: Index(['Center_ID', 'Requested_Demand_Tons', 'Region'], dtype='object')
Farms Dataset Columns: Index(['Farm_ID', 'Bio_Material_Capacity_Tons', 'Quality', 'Cost_Per_Ton',
       'Transport_Cost_To_Plant_1', 'Transport_Cost_To_Plant_2',
       'Transport_Cost_To_Plant_3', 'Transport_Cost_To_Plant_4',
       'Transport_Cost_To_Plant_5', 'Transport_Cost_To_Plant_6',
       'Transport_Cost_To_Plant_7', 'Transport_Cost_To_Plant_8',
       'Transport_Cost_To_Plant_9', 'Transport_Cost_To_Plant_10',
       'Transport_Cost_To_Plant_11', 'Transport_Cost_To_Plant_12',
       'Transport_Cost_To_Plant_13', 'Transport_Cost_To_Plant_14',
       'Transport_Cost_To_Plant_15', 'Transport_Cost_To_Plant_16',
       'Transport_Cost_To_Plant_17', 'Transport_Cost_To_Plant_18'],
      dtype='object')
Processing Dataset Columns: Index(['Processing_Plant_ID', 'Region', 'Capacity_Tons',
       'Processing_Cost_Per_Ton', 'Transport_Cost_To_Center_1',
       'Transport_Cost_To_Center_2', 'T

In [11]:
# Ensure column names are stripped of extra spaces
centers_data.columns = centers_data.columns.str.strip()
farms_data.columns = farms_data.columns.str.strip()
processing_data.columns = processing_data.columns.str.strip()

In [12]:
# Ensure expected column names exist
expected_processing_columns = ["Processing_Plant_ID", "Region", "Capacity_Tons", "Processing_Cost_Per_Ton"]
expected_centers_columns = ["Center_ID", "Requested_Demand_Tons", "Region"]
expected_farms_columns = ["Farm_ID", "Bio_Material_Capacity_Tons", "Quality", "Cost_Per_Ton"]

In [13]:
# Check for missing columns
missing_processing_cols = [col for col in expected_processing_columns if col not in processing_data.columns]
missing_centers_cols = [col for col in expected_centers_columns if col not in centers_data.columns]
missing_farms_cols = [col for col in expected_farms_columns if col not in farms_data.columns]

In [14]:
print("Missing columns in processing_data:", missing_processing_cols)
print("Missing columns in centers_data:", missing_centers_cols)
print("Missing columns in farms_data:", missing_farms_cols)

Missing columns in processing_data: []
Missing columns in centers_data: []
Missing columns in farms_data: []


In [15]:
# If all required columns exist, proceed with optimization
if not missing_processing_cols and not missing_centers_cols and not missing_farms_cols:
    # Extract unique regions for filtering
    processing_regions = processing_data.set_index("Processing_Plant_ID")["Region"].to_dict()
    center_regions = centers_data.set_index("Center_ID")["Region"].to_dict()

    # Define the optimization problem
    model = LpProblem("Optimal_Fertilizer_Distribution", LpMinimize)

    # Decision Variables
    # Transport from farms to processing plants
    farm_to_plant_vars = {
        (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
        for farm in farms_data["Farm_ID"]
        for plant in processing_data["Processing_Plant_ID"]
    }

    # Transport from processing plants to home centers (WITH REGIONAL RESTRICTION)
    plant_to_center_vars = {
        (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
        for plant in processing_data["Processing_Plant_ID"]
        for center in centers_data["Center_ID"]
        if processing_regions.get(plant) == center_regions.get(center)  # Apply regional restriction
    }

    # Objective Function: Minimize Total Cost
    model += lpSum(
        farm_to_plant_vars[farm, plant] * farms_data.loc[farms_data["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
        + farms_data.loc[farms_data["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
        for farm, plant in farm_to_plant_vars
    ) + lpSum(
        plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
        + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
        for plant, center in plant_to_center_vars
    )

    # Constraints

    # Supply constraint: Farms cannot supply more than their capacity
    for farm in farms_data["Farm_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= farms_data.loc[farms_data["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

    # Processing Capacity Constraint: Plants cannot process more than capacity
    for plant in processing_data["Processing_Plant_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) <= processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0]

    # Demand Constraint: Home centers must receive exactly their required demand
    for center in centers_data["Center_ID"]:
        model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"] if (plant, center) in plant_to_center_vars) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

    # Flow Constraint: Total input to a processing plant must match the total output from the plant
    for plant in processing_data["Processing_Plant_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) == lpSum(plant_to_center_vars[plant, center] for center in centers_data["Center_ID"] if (plant, center) in plant_to_center_vars)

    # Solve the model
    model.solve()

    # Extract optimal cost
    optimal_cost = model.objective.value()
    optimal_cost
else:
    print("Error: Some required columns are missing. Please check dataset column names.")

# Part d

In [16]:
# Ensure column names are stripped of extra spaces
centers_data.columns = centers_data.columns.str.strip()
farms_data.columns = farms_data.columns.str.strip()
processing_data.columns = processing_data.columns.str.strip()

# Filter farms to include only those with quality levels 3 and 4
high_quality_farms = farms_data[farms_data["Quality"].isin([3, 4])]

# Define the optimization problem with high-quality raw material sourcing
model = LpProblem("Optimal_Fertilizer_Cost_High_Quality", LpMinimize)

# Decision Variables
farm_to_plant_vars = {
    (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
    for farm in high_quality_farms["Farm_ID"]
    for plant in processing_data["Processing_Plant_ID"]
}

plant_to_center_vars = {
    (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
    for plant in processing_data["Processing_Plant_ID"]
    for center in centers_data["Center_ID"]
}

# Objective Function: Minimize Total Cost
model += lpSum(
    farm_to_plant_vars[farm, plant] * high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
    + high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
    for farm, plant in farm_to_plant_vars
) + lpSum(
    plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
    + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
    for plant, center in plant_to_center_vars
)

# Constraints

# Supply constraint: Farms cannot supply more than their capacity
for farm in high_quality_farms["Farm_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

# Processing Capacity Constraint: Plants cannot process more than capacity
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) <= processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0]

# Demand Constraint: Home centers must receive exactly their required demand
for center in centers_data["Center_ID"]:
    model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"]) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

# Flow Constraint: Total input to a processing plant must match the total output from the plant
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) == lpSum(plant_to_center_vars[plant, center] for center in centers_data["Center_ID"])

# Solve the model
model.solve()

# Extract optimal cost when using only high-quality raw materials
optimal_cost_high_quality = model.objective.value()
print(f"Optimal Cost of High Quality Materials: {optimal_cost_high_quality}")


Optimal Cost of High Quality Materials: 360401.7033432539


# Part e

In [17]:
# Total raw material sourced from farms
total_raw_material = farms_data["Bio_Material_Capacity_Tons"].sum()

# Define the optimization problem with new constraints
model = LpProblem("Optimal_Fertilizer_Distribution_With_Risk_Mitigation", LpMinimize)

# Decision Variables
# Transport from farms to processing plants
farm_to_plant_vars = {
    (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
    for farm in farms_data["Farm_ID"]
    for plant in processing_data["Processing_Plant_ID"]
}

# Transport from processing plants to home centers
plant_to_center_vars = {
    (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
    for plant in processing_data["Processing_Plant_ID"]
    for center in centers_data["Center_ID"]
}

# Objective Function: Minimize Total Cost
model += lpSum(
    farm_to_plant_vars[farm, plant] * farms_data.loc[farms_data["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
    + farms_data.loc[farms_data["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
    for farm, plant in farm_to_plant_vars
) + lpSum(
    plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
    + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
    for plant, center in plant_to_center_vars
)

# Constraints

# Supply constraint: Farms cannot supply more than their capacity
for farm in farms_data["Farm_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= farms_data.loc[farms_data["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

# Processing Capacity Constraint: Plants cannot process more than capacity and no more than 3% of all raw material
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) <= min(
        processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0],
        0.03 * total_raw_material  # 3% sourcing risk mitigation
    )

# Demand Constraint: Home centers must receive exactly their required demand
for center in centers_data["Center_ID"]:
    model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"]) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

# Supply risk mitigation: A single plant cannot supply more than 50% of a home center's demand
for center in centers_data["Center_ID"]:
    demand = centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]
    for plant in processing_data["Processing_Plant_ID"]:
        model += plant_to_center_vars[plant, center] <= 0.5 * demand

# Flow Constraint: Total input to a processing plant must match the total output from the plant
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) == lpSum(plant_to_center_vars[plant, center] for center in centers_data["Center_ID"])

# Solve the model
model.solve()

# Extract optimal cost
optimal_cost_risk_mitigation = model.objective.value()

print(f"Optimal Cost of Risk Mitigation: {optimal_cost_risk_mitigation}")

Optimal Cost of Risk Mitigation: 670462.8714653344


# Part f

In [18]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Ensure column names are stripped of extra spaces
centers_data.columns = centers_data.columns.str.strip()
farms_data.columns = farms_data.columns.str.strip()
processing_data.columns = processing_data.columns.str.strip()

# Total raw material sourced from farms
total_raw_material = farms_data["Bio_Material_Capacity_Tons"].sum()

# Extract unique regions for filtering (Regional Restriction from part c)
processing_regions = processing_data.set_index("Processing_Plant_ID")["Region"].to_dict()
center_regions = centers_data.set_index("Center_ID")["Region"].to_dict()

# Filter farms to include only those with quality levels 3 and 4 (High-Quality from part d)
high_quality_farms = farms_data[farms_data["Quality"].isin([3, 4])]

# Define the optimization problem
model = LpProblem("Optimal_Fertilizer_Distribution_All_Constraints", LpMinimize)

# Decision Variables
# Transport from farms to processing plants
farm_to_plant_vars = {
    (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
    for farm in high_quality_farms["Farm_ID"]  # Only high-quality farms
    for plant in processing_data["Processing_Plant_ID"]
}

# Transport from processing plants to home centers with regional restriction
plant_to_center_vars = {
    (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
    for plant in processing_data["Processing_Plant_ID"]
    for center in centers_data["Center_ID"]
    if processing_regions.get(plant) == center_regions.get(center)  # Regional Restriction
}

# Objective Function: Minimize Total Cost
model += lpSum(
    farm_to_plant_vars[farm, plant] * high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
    + high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
    for farm, plant in farm_to_plant_vars
) + lpSum(
    plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
    + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
    for plant, center in plant_to_center_vars
)

# Constraints

# 1. Supply Constraint: Farms cannot supply more than their capacity
for farm in high_quality_farms["Farm_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

# 2. Processing Capacity Constraint: Plants cannot process more than capacity or 3% of total raw material
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) <= min(
        processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0],
        0.03 * total_raw_material  # 3% sourcing risk mitigation
    )

# 3. Demand Constraint: Home centers must receive exactly their required demand
for center in centers_data["Center_ID"]:
    model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"] if (plant, center) in plant_to_center_vars) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

# 4. Supply Risk Mitigation: A single plant cannot supply more than 50% of a home center's demand
for center in centers_data["Center_ID"]:
    demand = centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]
    for plant in processing_data["Processing_Plant_ID"]:
        if (plant, center) in plant_to_center_vars:
            model += plant_to_center_vars[plant, center] <= 0.5 * demand

# 5. Flow Constraint: Total input to a processing plant must match the total output from the plant
for plant in processing_data["Processing_Plant_ID"]:
    model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) == lpSum(
        plant_to_center_vars[plant, center] for center in centers_data["Center_ID"] if (plant, center) in plant_to_center_vars
    )

# Solve the model
model.solve()

# Extract the optimal cost
optimal_cost_all_constraints = model.objective.value()
print(f"Optimal Cost with All Constraints: {optimal_cost_all_constraints}")


Optimal Cost with All Constraints: 366841.0160312151


# Part h

In [19]:
from pulp import LpStatus

# Define a function to test feasibility with varying sourcing risk mitigation percentages
def test_risk_mitigation_threshold():
    # Initialize variables
    step = 0.1  # Step to reduce the sourcing percentage (0.1% at a time)
    sourcing_risk_percentage = 3.0  # Start with 3%
    infeasible = False

    # Iterate while the model remains feasible
    while sourcing_risk_percentage > 0:
        # Define the optimization problem
        model = LpProblem(f"Risk_Mitigation_{sourcing_risk_percentage:.1f}_Percent", LpMinimize)

        # Decision Variables
        farm_to_plant_vars = {
            (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
            for farm in high_quality_farms["Farm_ID"]  # High-quality farms
            for plant in processing_data["Processing_Plant_ID"]
        }

        plant_to_center_vars = {
            (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
            for plant in processing_data["Processing_Plant_ID"]
            for center in centers_data["Center_ID"]
            if processing_regions.get(plant) == center_regions.get(center)  # Regional restriction
        }

        # Objective Function: Minimize Total Cost
        model += lpSum(
            farm_to_plant_vars[farm, plant] * high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
            + high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
            for farm, plant in farm_to_plant_vars
        ) + lpSum(
            plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
            + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
            for plant, center in plant_to_center_vars
        )

        # Constraints

        # 1. Supply Constraint
        for farm in high_quality_farms["Farm_ID"]:
            model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= high_quality_farms.loc[high_quality_farms["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

        # 2. Processing Capacity Constraint
        for plant in processing_data["Processing_Plant_ID"]:
            model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) <= min(
                processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0],
                (sourcing_risk_percentage / 100) * total_raw_material  # Adjusted sourcing risk mitigation
            )

        # 3. Demand Constraint
        for center in centers_data["Center_ID"]:
            model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"] if (plant, center) in plant_to_center_vars) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

        # 4. Supply Risk Mitigation Constraint
        for center in centers_data["Center_ID"]:
            demand = centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]
            for plant in processing_data["Processing_Plant_ID"]:
                if (plant, center) in plant_to_center_vars:
                    model += plant_to_center_vars[plant, center] <= 0.5 * demand

        # 5. Flow Constraint
        for plant in processing_data["Processing_Plant_ID"]:
            model += lpSum(farm_to_plant_vars[farm, plant] for farm in high_quality_farms["Farm_ID"]) == lpSum(
                plant_to_center_vars[plant, center] for center in centers_data["Center_ID"] if (plant, center) in plant_to_center_vars
            )

        # Solve the model
        model.solve()

        # Check feasibility
        if LpStatus[model.status] != "Optimal":
            infeasible = True
            print(f"Model becomes infeasible at sourcing risk mitigation percentage: {sourcing_risk_percentage:.1f}%")
            break

        # Reduce the sourcing risk percentage
        sourcing_risk_percentage -= step

    # If the model remains feasible down to 0%
    if not infeasible:
        print("Model remains feasible even with 0% sourcing risk mitigation.")

# Run the function to find the infeasibility threshold
test_risk_mitigation_threshold()


Model becomes infeasible at sourcing risk mitigation percentage: 2.4%


In [20]:
# Ensure column names are stripped of extra spaces
centers_data.columns = centers_data.columns.str.strip()
farms_data.columns = farms_data.columns.str.strip()
processing_data.columns = processing_data.columns.str.strip()

# Total raw material sourced from farms
total_raw_material = farms_data["Bio_Material_Capacity_Tons"].sum()

# Find the lowest feasible sourcing risk percentage
sourcing_risk_percentages = [2.9, 2.8, 2.7, 2.6, 2.5, 2.4, 2.3, 2.2, 2.1, 2.0, 1.9, 1.8, 1.7, 1.6, 1.5]  # Decreasing in steps of 0.1%
infeasible_threshold = None

for percent in sourcing_risk_percentages:
    # Define the optimization problem with decreasing sourcing risk mitigation percentage
    model = LpProblem("Sourcing_Risk_Limit_Test", LpMinimize)

    # Decision Variables
    farm_to_plant_vars = {
        (farm, plant): LpVariable(f"Transport_{farm}_{plant}", lowBound=0)
        for farm in farms_data["Farm_ID"]
        for plant in processing_data["Processing_Plant_ID"]
    }

    plant_to_center_vars = {
        (plant, center): LpVariable(f"Transport_{plant}_{center}", lowBound=0)
        for plant in processing_data["Processing_Plant_ID"]
        for center in centers_data["Center_ID"]
    }

    # Objective Function: Minimize Total Cost
    model += lpSum(
        farm_to_plant_vars[farm, plant] * farms_data.loc[farms_data["Farm_ID"] == farm, f"Transport_Cost_To_{plant}"].values[0]
        + farms_data.loc[farms_data["Farm_ID"] == farm, "Cost_Per_Ton"].values[0]
        for farm, plant in farm_to_plant_vars
    ) + lpSum(
        plant_to_center_vars[plant, center] * processing_data.loc[processing_data["Processing_Plant_ID"] == plant, f"Transport_Cost_To_{center}"].values[0]
        + processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Processing_Cost_Per_Ton"].values[0]
        for plant, center in plant_to_center_vars
    )

    # Constraints

    # Supply constraint: Farms cannot supply more than their capacity
    for farm in farms_data["Farm_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for plant in processing_data["Processing_Plant_ID"]) <= farms_data.loc[farms_data["Farm_ID"] == farm, "Bio_Material_Capacity_Tons"].values[0]

    # Processing Capacity Constraint: Plants cannot process more than capacity and no more than 'percent' of total raw material
    for plant in processing_data["Processing_Plant_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) <= min(
            processing_data.loc[processing_data["Processing_Plant_ID"] == plant, "Capacity_Tons"].values[0],
            (percent / 100) * total_raw_material  # Decreasing sourcing risk mitigation
        )

    # Demand Constraint: Home centers must receive exactly their required demand
    for center in centers_data["Center_ID"]:
        model += lpSum(plant_to_center_vars[plant, center] for plant in processing_data["Processing_Plant_ID"]) == centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]

    # Supply risk mitigation: A single plant cannot supply more than 50% of a home center's demand
    for center in centers_data["Center_ID"]:
        demand = centers_data.loc[centers_data["Center_ID"] == center, "Requested_Demand_Tons"].values[0]
        for plant in processing_data["Processing_Plant_ID"]:
            model += plant_to_center_vars[plant, center] <= 0.5 * demand

    # Flow Constraint: Total input to a processing plant must match the total output from the plant
    for plant in processing_data["Processing_Plant_ID"]:
        model += lpSum(farm_to_plant_vars[farm, plant] for farm in farms_data["Farm_ID"]) == lpSum(plant_to_center_vars[plant, center] for center in centers_data["Center_ID"])

    # Solve the model
    model.solve()

    # Check feasibility
    if LpStatus[model.status] == "Infeasible":
        infeasible_threshold = percent
        break  # Stop when first infeasible percentage is found

infeasible_threshold


1.5

# Question 2

# Part c

In [21]:
df = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/updated_gym_data.csv')

In [22]:
from gurobipy import Model, GRB

# Initialize the optimization model
model = Model("Optimal_Hypertrophy")

# Extract necessary data from the dataset
exercises = df.index.tolist()  # Exercise indices (0 to 2636)
hypertrophy = df["Hypertrophy Rating"].tolist()  # Hypertrophy ratings
sfr = df["Stimulus-to-Fatigue"].tolist()  # SFR values
body_part = df["BodyPart"].tolist()  # Target muscle groups
difficulty = df["Difficulty"].tolist()  # Exercise difficulty levels
category = df["Category"].tolist()  # Training category (Powerlifting, Strongman, etc.)
equipment = df["Equipment"].fillna("Other").tolist()  # Fill missing equipment values


In [23]:
# Define decision variables (exercise proportions)
x = model.addVars(exercises, lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective Function: Maximize Hypertrophy Rating
model.setObjective(sum(hypertrophy[i] * x[i] for i in exercises), GRB.MAXIMIZE)

# Constraint 1: Total Allocation Must Sum to 1 (100% of the program)
model.addConstr(sum(x[i] for i in exercises) == 1, "Total_Allocation")

# Constraint 2: Body Part Allocation Constraints
body_part_constraints = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}

# Ensure each required body part gets at least its minimum allocation
for part, min_alloc in body_part_constraints.items():
    model.addConstr(sum(x[i] for i in exercises if body_part[i] == part) >= min_alloc, f"Min_{part}")

# General muscle groups should be at least 2.5%
muscle_groups = set(body_part) - set(body_part_constraints.keys())
for group in muscle_groups:
    model.addConstr(sum(x[i] for i in exercises if body_part[i] == group) >= 0.025, f"Min_{group}")

# Constraint 3: Leg Muscles Receive 2.6× Upper Body Allocation
leg_muscles = ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]
upper_body_muscles = ["Biceps", "Triceps", "Chest", "Upper Back", "Lower Back", "Lats"]

leg_alloc = sum(x[i] for i in exercises if body_part[i] in leg_muscles)
upper_body_alloc = sum(x[i] for i in exercises if body_part[i] in upper_body_muscles)
model.addConstr(leg_alloc >= 2.6 * upper_body_alloc, "Leg_vs_UpperBody")

# Constraint 4: Biceps = Triceps & Chest = Back
biceps_alloc = sum(x[i] for i in exercises if body_part[i] == "Biceps")
triceps_alloc = sum(x[i] for i in exercises if body_part[i] == "Triceps")
model.addConstr(biceps_alloc == triceps_alloc, "Biceps_Equal_Triceps")

chest_alloc = sum(x[i] for i in exercises if body_part[i] == "Chest")
back_alloc = sum(x[i] for i in exercises if body_part[i] in ["Upper Back", "Lower Back", "Lats"])
model.addConstr(chest_alloc == back_alloc, "Chest_Equal_Back")

# Constraint 5: Overall Stimulus-to-Fatigue Ratio ≤ 0.55
model.addConstr(sum(sfr[i] * x[i] for i in exercises) <= 0.55, "SFR_Limit")

# Constraint 6: Beginner ≥ 1.4× Intermediate, Intermediate ≥ 1.1× Advanced
beginner_alloc = sum(x[i] for i in exercises if difficulty[i] == "Beginner")
intermediate_alloc = sum(x[i] for i in exercises if difficulty[i] == "Intermediate")
advanced_alloc = sum(x[i] for i in exercises if difficulty[i] == "Advanced")

model.addConstr(beginner_alloc >= 1.4 * intermediate_alloc, "Beginner_vs_Intermediate")
model.addConstr(intermediate_alloc >= 1.1 * advanced_alloc, "Intermediate_vs_Advanced")

# Constraint 7: Training Style Allocations
strongman_alloc = sum(x[i] for i in exercises if category[i] == "Strongman")
powerlifting_alloc = sum(x[i] for i in exercises if category[i] == "Powerlifting")
olympic_alloc = sum(x[i] for i in exercises if category[i] == "Olympic Weightlifting")

model.addConstr(strongman_alloc <= 0.08, "Strongman_Limit")
model.addConstr(powerlifting_alloc >= 0.09, "Powerlifting_Min")
model.addConstr(olympic_alloc >= 0.10, "Olympic_Min")

# Constraint 8: Equipment Allocation ≥ 60%
selected_equipment = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_alloc = sum(x[i] for i in exercises if any(eq in equipment[i] for eq in selected_equipment))
model.addConstr(equipment_alloc >= 0.60, "Equipment_Min")


<gurobi.Constr *Awaiting Model Update*>

In [24]:
# Solve the optimization problem
model.optimize()

# Extract and display the optimal hypertrophy rating
optimal_hypertrophy = model.objVal if model.status == GRB.OPTIMAL else None
optimal_hypertrophy


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 28 rows, 2637 columns and 16472 nonzeros
Model fingerprint: 0xb635e1ff
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 27 rows, 2637 columns, 14198 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      38    7.6660594e-01   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.03 seconds (0.01 work units)
Optimal objective  7.666059373e-01


0.7666059373071197

# Part d

In [25]:
df = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/updated_gym_data.csv')

In [26]:
import pandas as pd
from scipy.optimize import linprog

In [27]:

# Define the objective function coefficients (Hypertrophy Rating)
c = [-hr for hr in df["Hypertrophy Rating"].tolist()]  # Negative for maximization

# Define inequality constraints (A_ub * x <= b_ub)
A_ub = []  # Coefficients for inequality constraints
b_ub = []  # Upper bounds

# Constraint 1: Total Allocation Must Sum to 1 (Equality constraint)
A_eq = [[1] * len(df)]  # Sum of all exercise proportions must equal 1
b_eq = [1]

# Constraint 2: Upper bound of each exercise proportion (≤ 0.05)
bounds = [(0, 0.05) for _ in range(len(df))]

# Constraint 5: Stimulus-to-Fatigue Ratio (Original: SFR ≤ 0.55)
A_ub.append(df["Stimulus-to-Fatigue"].tolist())
b_ub.append(0.55)

# Solve the original LP
result_original = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

# Constraint 5 Relaxed: SFR ≤ 0.551
b_ub[-1] = 0.551  # Increase the constraint limit by 0.001

# Solve the relaxed LP
result_relaxed = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

# Calculate the improvement in hypertrophy rating
improvement = -result_relaxed.fun - (-result_original.fun) if result_original.success and result_relaxed.success else None

# Check validity: Use dual values (shadow price) if available
shadow_price = result_original.con if result_original.success else None

improvement, shadow_price


(0.00015316681088295425, array([0.]))

# Part e

In [28]:
# Extract the optimal exercise allocation from the previous optimization
optimal_allocations = result_original.x if result_original.success else None

# If optimization was successful, compute total allocations for the specified body parts
if optimal_allocations is not None:
    body_part_allocations = {
        "Traps": sum(optimal_allocations[i] for i in range(len(df)) if df["BodyPart"][i] == "Traps"),
        "Neck": sum(optimal_allocations[i] for i in range(len(df)) if df["BodyPart"][i] == "Neck"),
        "Forearms": sum(optimal_allocations[i] for i in range(len(df)) if df["BodyPart"][i] == "Forearms"),
        "Abdominals": sum(optimal_allocations[i] for i in range(len(df)) if df["BodyPart"][i] == "Abdominals"),
    }
else:
    body_part_allocations = None

body_part_allocations


{'Traps': 0.0, 'Neck': 0.0, 'Forearms': 0.0, 'Abdominals': 0.0}

In [29]:
# Increase the minimum required allocation for Traps, Neck, Forearms, and Abdominals
increased_body_part_constraints = {
    "Traps": 0.01,  # Increase from 0.005 to 0.01 (1%)
    "Neck": 0.01,   # Increase from 0.005 to 0.01 (1%)
    "Forearms": 0.01,  # Increase from 0.005 to 0.01 (1%)
    "Abdominals": 0.05  # Increase from 0.04 to 0.05 (5%)
}

# Update the constraints in the model
A_ub_new = A_ub.copy()
b_ub_new = b_ub.copy()

for part, new_min in increased_body_part_constraints.items():
    A_ub_new.append([1 if df["BodyPart"][i] == part else 0 for i in range(len(df))])
    b_ub_new.append(new_min)

# Solve the new LP with increased allocations
result_increased = linprog(c, A_ub=A_ub_new, b_ub=b_ub_new, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

# Compare hypertrophy rating before and after
hypertrophy_improvement = -result_increased.fun - (-result_original.fun) if result_original.success and result_increased.success else None

hypertrophy_improvement


0.00015316681088295425

# Part f

In [30]:
# loading dataset
gym_data_path = ('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/updated_gym_data.csv')
gym_data = pd.read_csv(gym_data_path)

In [31]:
# Extract index of Barbell Back Squats
barbell_squats_index = gym_data[gym_data['Exercise'] == 'Barbell Back Squats'].index[0]

# Copy the original hypertrophy rating
original_hypertrophy_rating = gym_data.loc[barbell_squats_index, 'Hypertrophy Rating']


In [32]:
# Example gym_data DataFrame
import pandas as pd
gym_data = pd.DataFrame({
    'Exercise': ['Barbell Back Squats', 'Bench Press', 'Deadlift'],
    'Hypertrophy Rating': [0.5, 0.7, 0.6]
})

# Define exercises as a list of indices
exercises = gym_data.index.tolist()


In [33]:
barbell_squats_index = gym_data[gym_data['Exercise'] == 'Barbell Back Squats'].index[0]


In [34]:
# Create the Gurobi model
model = Model("Hypertrophy_Optimization")

# Add decision variables for each exercise
x = model.addVars(exercises, vtype=GRB.BINARY, name="x")

# Add constraints (example)
model.addConstrs(x[i] <= 1 for i in exercises)  # Binary decision variables

# Set the initial objective function
model.setObjective(
    quicksum(x[i] * gym_data.loc[i, 'Hypertrophy Rating'] for i in exercises),
    GRB.MAXIMIZE
)

# Optimize the initial model
model.optimize()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x1c7a0b7f
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.8000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.800000000000e+00, best bound 1.800000000000e+00, gap 0.0000%


In [35]:
# Ensure consistent indexing in gym_data
gym_data.reset_index(drop=True, inplace=True)

# Find the index for Barbell Back Squats
if "Barbell Back Squats" in gym_data['Exercise'].values:
    barbell_squats_index = gym_data[gym_data['Exercise'] == 'Barbell Back Squats'].index[0]
else:
    raise ValueError("Barbell Back Squats not found in gym_data.")

# Sensitivity analysis: Increment hypertrophy rating until included
increment = 0.01
max_hypertrophy = 1.0  # Upper limit for hypertrophy rating
included = False

while not included and gym_data.loc[barbell_squats_index, 'Hypertrophy Rating'] <= max_hypertrophy:
    # Update hypertrophy rating
    gym_data.loc[barbell_squats_index, 'Hypertrophy Rating'] += increment

    # Update objective function with the new hypertrophy ratings
    model.setObjective(
        quicksum(x[i] * gym_data.loc[i, 'Hypertrophy Rating'] for i in exercises),
        GRB.MAXIMIZE
    )

    # Re-optimize
    model.optimize()

    # Check if Barbell Back Squats is included
    if x[barbell_squats_index].x > 0:
        included = True
        print(f"Barbell Back Squats included with hypertrophy rating: {gym_data.loc[barbell_squats_index, 'Hypertrophy Rating']}")

if not included:
    print("Barbell Back Squats cannot be included, even with maximum hypertrophy rating.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x2393b0cc
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded MIP start from previous solve with objective 1.81


Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1.81 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.810000000000e+00, best bound 1.810000000000e+00, gap 0.0000%
Barbell Back Squats included with hypertrophy rating: 0.51


# Part g

In [36]:
# loading dataset
gym_data_path = ('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/updated_gym_data.csv')
gym_data = pd.read_csv(gym_data_path)


In [37]:
# Verify the column names
print("Columns in the dataset:", gym_data.columns)

# Ensure the 'Equipment' column exists and is correctly referenced
standard_equipment = ['Barbells', 'Dumbbells', 'Machines', 'Cables', 'Bands', 'E-Z Curl Bar']
if 'Equipment' in gym_data.columns:
    equipment_allocation = sum(
        x[i].x for i in exercises if gym_data.loc[i, 'Equipment'] in standard_equipment
    )
    print(f"\nTotal Allocation to Standard Equipment: {equipment_allocation:.4f}")
else:
    print("The 'Equipment' column is missing or improperly named in the dataset.")


Columns in the dataset: Index(['Exercise', 'Category', 'BodyPart', 'Equipment', 'Difficulty',
       'Stimulus-to-Fatigue', 'Expected Time', 'Hypertrophy Rating'],
      dtype='object')

Total Allocation to Standard Equipment: 3.0000


In [38]:
# Analyze the optimal workout program
if model.status == GRB.OPTIMAL:
    print("Optimal Workout Program Allocations:")
    for i in exercises:
        if x[i].x > 0:
            print(f"{gym_data.loc[i, 'Exercise']}: {x[i].x:.4f}")
    
    # Analyze total allocation to standard equipment
    standard_equipment = ['Barbells', 'Dumbbells', 'Machines', 'Cables', 'Bands', 'E-Z Curl Bar']
    equipment_allocation = sum(x[i].x for i in exercises if gym_data.loc[i, 'Equipment'] in standard_equipment)
    print(f"\nTotal Allocation to Standard Equipment: {equipment_allocation:.4f}")

    # Analyze allocation by body part
    body_part_allocations = gym_data.groupby('BodyPart')['Hypertrophy Rating'].sum()
    print("\nBody Part Allocations:")
    print(body_part_allocations)


Optimal Workout Program Allocations:
Bench Press With Short Bands: 1.0000
Hip Lift with Band: 1.0000
Band Good Morning (Pull Through): 1.0000

Total Allocation to Standard Equipment: 3.0000

Body Part Allocations:
BodyPart
Abdominals     344.378898
Abductors        6.072283
Adductors        2.934949
Biceps          96.622704
Calves          18.027344
Chest          132.737954
Forearms        14.817048
Glutes          38.991606
Hamstrings      48.971870
Lats            61.983604
Lower Back      49.383049
Middle Back     64.560334
Neck             2.298934
Quadriceps     308.128500
Shoulders      182.460707
Traps           13.825678
Triceps         85.855842
Name: Hypertrophy Rating, dtype: float64


# Part h

In [39]:
# Reading File data
df = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/operational_reserach/refs/heads/main/updated_gym_data.csv')

In [40]:
from gurobipy import Model, GRB

# Initialize the optimization model
model = Model("Optimal_Hypertrophy")

# Extract necessary data from the dataset
exercises = df.index.tolist()  # Exercise indices (0 to 2636)
hypertrophy = df["Hypertrophy Rating"].tolist()  # Hypertrophy ratings
sfr = df["Stimulus-to-Fatigue"].tolist()  # SFR values
body_part = df["BodyPart"].tolist()  # Target muscle groups
difficulty = df["Difficulty"].tolist()  # Exercise difficulty levels
category = df["Category"].tolist()  # Training category (Powerlifting, Strongman, etc.)
equipment = df["Equipment"].fillna("Other").tolist()  # Fill missing equipment values

In [41]:
# Define decision variables (exercise proportions)
x = model.addVars(exercises, lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective Function: Maximize Hypertrophy Rating
model.setObjective(sum(hypertrophy[i] * x[i] for i in exercises), GRB.MAXIMIZE)

# Constraint 1: Total Allocation Must Sum to 1 (100% of the program)
model.addConstr(sum(x[i] for i in exercises) == 1, "Total_Allocation")

# Constraint 2: Body Part Allocation Constraints
body_part_constraints = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}

# Ensure each required body part gets at least its minimum allocation
for part, min_alloc in body_part_constraints.items():
    model.addConstr(sum(x[i] for i in exercises if body_part[i] == part) >= min_alloc, f"Min_{part}")

# General muscle groups should be at least 2.5%
muscle_groups = set(body_part) - set(body_part_constraints.keys())
for group in muscle_groups:
    model.addConstr(sum(x[i] for i in exercises if body_part[i] == group) >= 0.025, f"Min_{group}")


# Constraint 8: Equipment Allocation ≥ 60%
selected_equipment = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_alloc = sum(x[i] for i in exercises if any(eq in equipment[i] for eq in selected_equipment))
model.addConstr(equipment_alloc >= 0.60, "Equipment_Min")


<gurobi.Constr *Awaiting Model Update*>

In [42]:
# Solve the optimization problem
model.optimize()

# Extract and display the optimal hypertrophy rating
optimal_hypertrophy = model.objVal if model.status == GRB.OPTIMAL else None
optimal_hypertrophy


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 19 rows, 2637 columns and 6580 nonzeros
Model fingerprint: 0x58e61afe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.01s
Presolved: 19 rows, 2637 columns, 6580 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   1.305000e+00   0.000000e+00      0s
      14    8.5703576e-01   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.570357586e-01


0.8570357585549997

# Part i

In [43]:
# Initialize the primal model
primal_model = Model("Hypertrophy Optimization")

In [44]:
# Primal Decision variables: proportion of each exercise in the workout program
primal_vars = primal_model.addVars(len(ratings), lb=0, ub=1, name="x")

# Primal Objective: Maximize the total hypertrophy rating
primal_model.setObjective(sum(primal_vars[i] * ratings[i] for i in range(len(ratings))), GRB.MAXIMIZE)


NameError: name 'ratings' is not defined

In [39]:
# Primal Constraint 1: No single exercise > 5% of total
for i in range(len(ratings)):
    primal_model.addConstr(primal_vars[i] <= 0.05, name=f"max_exercise_{i}")

# Primal Constraint 2: Minimum allocation for each body part
body_part_min = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04,
    "default": 0.025  # Default for all other body parts
}
for body_part in body_parts:
    min_alloc = body_part_min.get(body_part, body_part_min["default"])
    primal_model.addConstr(
        sum(primal_vars[i] for i in range(len(ratings)) if gym_data.loc[i, "BodyPart"] == body_part) >= min_alloc,
        name=f"min_bodypart_{body_part}"
    )

# Primal Constraint 8: Equipment usage > 60%
equipment_mask = [1 if gym_data.loc[i, "Equipment"] in equipment_types else 0 for i in range(len(ratings))]
primal_model.addConstr(
    sum(primal_vars[i] * equipment_mask[i] for i in range(len(ratings))) >= 0.6,
    name="equipment_usage"
)

# Primal Constraint: Total allocation = 100%
primal_model.addConstr(sum(primal_vars[i] for i in range(len(ratings))) == 1.0, name="total_allocation")


<gurobi.Constr *Awaiting Model Update*>

In [40]:
# Optimize the primal model
primal_model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 20.6.0 20G95)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2656 rows, 2637 columns and 8008 nonzeros
Model fingerprint: 0xb2b84198
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2637 rows and 0 columns
Presolve time: 0.13s
Presolved: 19 rows, 2637 columns, 5371 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   1.905000e+00   0.000000e+00      0s
      13    7.1419485e-01   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.19 seconds (0.00 work units)
Optimal objective  7.141948453e-01


In [41]:
# Formulate and solve the dual problem
# Dual model initialization
dual_model = Model("Hypertrophy Dual Optimization")

# Dual variables
dual_max_exercise = dual_model.addVars(len(ratings), lb=0, name="dual_max_exercise")
dual_min_bodypart = dual_model.addVars(body_parts, lb=0, name="dual_min_bodypart")
dual_equipment = dual_model.addVar(lb=0, name="dual_equipment")
dual_total_allocation = dual_model.addVar(lb=0, name="dual_total_allocation")

# Dual Objective: Minimize the combined dual variables
dual_model.setObjective(
    sum(0.05 * dual_max_exercise[i] for i in range(len(ratings))) +
    sum(body_part_min.get(bp, body_part_min["default"]) * dual_min_bodypart[bp] for bp in body_parts) +
    0.6 * dual_equipment +
    dual_total_allocation,
    GRB.MINIMIZE
)

# Dual Constraints: Ensure each exercise's contribution is covered
for i in range(len(ratings)):
    exercise_body_part = gym_data.loc[i, "BodyPart"]
    exercise_equipment = 1 if gym_data.loc[i, "Equipment"] in equipment_types else 0

    dual_model.addConstr(
        0.05 * dual_max_exercise[i] +
        dual_min_bodypart[exercise_body_part] +
        exercise_equipment * dual_equipment +
        dual_total_allocation >= ratings[i],
        name=f"dual_constraint_{i}"
    )

# Optimize the dual model
dual_model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 20.6.0 20G95)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2637 rows, 2656 columns and 8008 nonzeros
Model fingerprint: 0xff82a557
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [5e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 2637 rows and 2656 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0684991e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  3.068499140e-01


In [42]:
# Extract dual results
if dual_model.status == GRB.OPTIMAL:
    optimal_dual_value = dual_model.objVal
    print(f"Optimal Dual Value: {optimal_dual_value:.4f}")
else:
    print("Dual optimization failed.")

Optimal Dual Value: 0.3068


# Part j

#### The primal model is generally easier to solve and interpret in this scenario because it directly provides the allocation of exercises to the workout program. However, the dual model might be preferred if:

#### 1. The primal has many variables but relatively few constraints.
#### 2. Insights into constraint sensitivities or resource allocations are needed.

#### For the dataset in this problem, where constraints (e.g., body parts, equipment) are fewer than the number of exercises, the dual model might be computationally simpler but harder to interpret directly in terms of exercise allocations.